In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import matplotlib.pyplot as plt
import scipy.misc
import sklearn.feature_extraction
import numpy as np
import scipy.ndimage
import scipy.signal
from skimage import measure, io
from skimage import transform
import train_UNET_2_input 
import skimage
import json
import datetime
import pickle
import tensorflow as tf
import cv2

epoch_cb_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Callback_images'
save_results = r'C:\Users\buggyr\Mosaic_Experiments\models'
load_training = r'C:\Users\buggyr\Mosaic_Experiments\data\external\Gharbi_tiled'
load_validation = r'C:\Users\buggyr\Mosaic_Experiments\data\external\Val_data'

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.models import *
from keras.models import Sequential
from keras.layers import (Conv2D, Activation, UpSampling2D,Reshape, MaxPooling2D,
Dropout, Cropping2D, merge, Input, concatenate, Conv2DTranspose, Lambda, Flatten,
                         Dense, LeakyReLU)
from keras.optimizers import Adadelta, Nadam, RMSprop, Adam
from keras.models import load_model

In [6]:
train_generator = train_UNET_2_input.train_generator_rgb_tiled(load_training, 64, 32)
validation_generator = train_UNET_2_input.train_generator_rgb_tiled(load_validation, 64, 32)

inputs, input_orig = next(train_generator)

good_gen_path = r'C:\Users\buggyr\Mosaic_Experiments\models\2018-02-07 20-22_UNET_2_layer_64x64_logcosh_normal1_Patterns+Gharbi_2_input_strides+upsample\Epoch_Models\model.111-0.00.hdf5'

model = keras.models.load_model(good_gen_path)

In [35]:
save_path = r'C:\Users\buggyr\Mosaic_Experiments\data\processed\Temp_generated_patches'
gen_batch = model.predict_on_batch(inputs)

for idx, i in enumerate(gen_batch):
    cv2.imwrite(os.path.join(save_path, 'generated_img'+str(idx)+'.png'), train_UNET_2_input.denormalise1(i))
    cv2.imwrite(os.path.join(save_path, 'Original_img'+str(idx)+'.png'), train_UNET_2_input.denormalise1(input_orig[idx]))
    print(skimage.measure.compare_psnr(input_orig[idx], gen_batch[idx], 1))

37.398763749686495
37.278940081434435
36.13289588229641
34.158073577292825
30.803020338334438
36.94382306992316
35.2468829551716
40.741387267387736
39.29195231812368
37.42747549779244
36.845164795079
27.300754090732873
36.97804769479694
28.59938382234215
38.938188211711605
35.69337894220558
39.000059221507556
38.57940588575236
39.52407426634142
36.92429193455412
35.37242864603273
36.03364977933878
38.835691138656955
33.27318735874907
39.3454937841463
39.33099097557927
34.73322594258191
35.64840352572537
33.55500703302033
28.565154855437175
29.073126109938254
39.4277722852399


In [44]:
fft = np.absolute(np.fft.fft2(gen_batch[18]))

train_UNET_2_input.show_image(fft)

In [129]:
f1 = matlab_style_gauss2D(shape = (128,128),sigma=0.5)
f1 = np.stack([f1, f1, f1], axis = -1)

fx = gen_batch[18]
fx1 = (fx - fx.mean(axis=(0,-2,-1), keepdims=1))*f1
y_pred = np.absolute(np.fft.fft2(fx1))

y = input_orig[18]
y_avg = np.mean(y, axis=-1)
y1 = (y - y.mean(axis=(0,-2,-1), keepdims=1))*f1
y_true = np.absolute(np.fft.fft2(y1))
            
mse = np.mean((y_true - y_pred)**2)
print('mse: ', str(mse))

mse:  6.510819245879872e-08


In [134]:
np.mean((fx - y)**2)

0.032037884

In [60]:
def matlab_style_gauss2D(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h


0.24108486166143514